# Guided Project: Answering Business Questions using SQL

In this guided project, we practice using our SQL skills to answer business questions. In each step, we need to:

- Write a SQL query to extract the relevant data
- Create plots where necessary to visualize the data
- Write a short paragraph, drawing conclusions and explaining the data and/or visualizations.

We will continue to use the Chinook database that we have previously used. The Chinook database is provided as a SQLite database file called `chinook.db`. A copy of the database schema is below:

![chinook db schema](chinook-schema.svg)

---

## Create Helper Functions

We will use Python's [SQLite module](https://docs.python.org/3/library/sqlite3.html) and Pandas to write queries that produced dataframe outputs. The advantage of dataframe outputs is that they will display as tables in Jupyter notebook. We can also take advantage of pandas plotting methods to easily create visualizations of our tables inside our Jupyter notebook.

The [visualization section of the pandas docs](https://pandas.pydata.org/pandas-docs/stable/visualization.html) will be useful - we encourage you to have a look and see if you can use a few plot types that you might not have already used before.

Previously, we used the [`sqlite3.connect()` function](https://docs.python.org/3/library/sqlite3.html#sqlite3.connect) to create a connection object, and passed that to the [`pandas.read_sql_query()` function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_query.html). In this project, we will use a similar approach, with two important changes:

- We create some helper functions in python to save some time.
- We use a **context manager** to handle the connection to the SQLite database.

Context managers make our life easy by taking care of opening and closing the connection to the database, as well as making sure we don't accidentally make changes to the database if one of our queries has an error. For learning more about context managers, read [this excellent article by Jeff Knupp](https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/). We use [the `with` statement](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement) to access the [context manager that's built-in to the SQLite module](https://docs.python.org/3/library/sqlite3.html#using-the-connection-as-a-context-manager):

```python
with sqlite3.connect('chinook.db') as conn:
    # we can now access `conn`
    # inside an indented block
    query_result = pd.read_sql(my_query, conn)
```

With a few extra lines, we can turn this code into a function:

```python
def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q, conn)
```

By returning the dataframe, we can just run our function as the last line of a Jupyter cell and it will print the results nicely for us.

We will also create a function to run SQL queries that don't return tables, like `CREATE VIEW`. In this instance, we don't need to use pandas, we can simply use the SQLite module's [`connection.execute()` method](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.execute):

```python
with sqlite3.connect('chinook.db') as conn:
    conn.isolation_level = None
    conn.execute(c)
```

The `conn.isolation_level = None` line above tells SQLite to autocommit any changes.

Lastly, we will make a function that uses our `run_query()` function to return a list of all tables and views in our databases - this is handy to quickly check the state of our database as we work:

```python
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");
```

In [1]:
# import libraries
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

/anaconda/envs/dataquest/lib/python3.7/site-packages/matplotlib/font_manager.py:232: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
def run_query(query):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(query, conn)

In [3]:
def run_command(command):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(command)

In [4]:
def show_tables():
    query = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table", "view");
    '''
    return run_query(query)

In [5]:
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


---

## Find Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and we have been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| **Artist Name** |	**Genre** |
|      :-         |    :-     |
| Regal	          | Hip-Hop   |
| Red Tone	      | Punk      |
| Meteor and the Girls | Pop  |
| Slim Jim Bites  | Blues     |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

We will need to write a query to find out which genres sell the most tracks in the USA, and then create a visualization of that data using pandas.

We can run the `run_query()` function twice. Once at the end of a cell to display the results, and then a second time to assign the output of yours to a variable name:

```python
genre_sales_usa = run_query(q)
```

We can then use the [`DataFrame.plot()` method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) or any of the other pandas plotting methods to produce your visualization.

Note that if you use the DataFrame.set_index() method before you create your plot, your axis labels will be automatically added to your plot as shown in this blogpost.

Lastly, we should write up a summary of our findings, and make a recommendation for the three artists whose albums we should purchase for the store.

In [6]:
# verification - know # of genres
q = '''
SELECT
    COUNT(DISTINCT(name)) num_genres
FROM genre;
'''
r = run_query(q)
r.head(10)

,num_genres
0,25


In [64]:
# Attempt 1 - FAILED - Retry!
# return each genre, with the number of tracks sold in 
# absolute numbers and in percentages

q1 = '''
WITH
    sales_per_track AS
        (
        SELECT
            t.track_id,
            t.name track_name,
            t.genre_id,
            CASE
                WHEN SUM(il.quantity) IS NULL THEN 0
                ELSE SUM(il.quantity)
                END
                AS track_sales
        FROM track t
        INNER JOIN invoice_line il ON il.track_id = t.track_id
        GROUP BY il.track_id
        ),
    sales_per_genre AS
        (
            SELECT
                g.name genre,
                g.genre_id,
                CASE
                    WHEN SUM(spt.track_sales) IS NULL THEN 0
                    ELSE SUM(spt.track_sales)
                    END
                    AS track_sales
            FROM genre g
            LEFT JOIN sales_per_track spt ON g.genre_id = spt.genre_id
            GROUP BY genre
        )

SELECT
    spg.genre,
    spg.genre_id,
    spg.track_sales,
    CAST(spg.track_sales AS FLOAT) / (SELECT COUNT(*) FROM sales_per_genre) 'track_sales_%'
FROM sales_per_genre spg
GROUP BY spg.genre_id;
'''
genre_sales_usa = run_query(q1)
genre_sales_usa.head(100)

,genre,genre_id,track_sales,track_sales_%
0,Rock,1,2635,105.40
1,Jazz,2,121,4.84
2,Metal,3,619,24.76
3,Alternative & Punk,4,492,19.68
4,Rock And Roll,5,0,0.00
5,Blues,6,124,4.96
6,Latin,7,167,6.68
7,Reggae,8,35,1.40
8,Pop,9,63,2.52
9,Soundtrack,10,5,0.20


In [66]:
# correct answer
albums_to_purchase = '''
WITH
    usa_tracks_sold AS
        (
            SELECT il.*
            FROM invoice_line il
            INNER JOIN invoice i ON il.invoice_id = i.invoice_id
            INNER JOIN customer c ON i.customer_id = c.customer_id
            WHERE c.country = 'USA'
        )

SELECT
    g.name genre,
    COUNT(uts.invoice_line_id) tracks_sold,
    CAST(COUNT(uts.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) FROM usa_tracks_sold
    ) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t ON t.track_id = uts.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 30;
'''
run_query(albums_to_purchase)

,genre,tracks_sold,percentage_sold
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Latin,22,0.020932
7,Pop,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


In [51]:
q = '''
SELECT il.*
FROM invoice_line il
INNER JOIN invoice i ON il.invoice_id = i.invoice_id
INNER JOIN customer c ON i.customer_id = c.customer_id
WHERE c.country = 'USA'
'''
r = run_query(q)
r.head(10)

,invoice_line_id,invoice_id,track_id,unit_price,quantity
0,1,1,1158,0.99,1
1,2,1,1159,0.99,1
2,3,1,1160,0.99,1
3,4,1,1161,0.99,1
4,5,1,1162,0.99,1
5,6,1,1163,0.99,1
6,7,1,1164,0.99,1
7,8,1,1165,0.99,1
8,9,1,1166,0.99,1
9,10,1,1167,0.99,1


---